In [122]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
import statsmodels.api as sm
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from statsmodels.sandbox.regression.gmm import IV2SLS
from linearmodels import PanelOLS, PooledOLS

In [123]:
import os
os.chdir('/Users/jhongyihuang/Downloads') 

In [124]:
df = pd.read_stata('blundell_bond_2000_production_function.dta')
df[["sales", "labor", "capital"]] = np.log(df[["sales", "labor", "capital"]])

In [125]:
df.head()

,id,year,sales,labor,capital
0,886.0,1982.0,4.579228,0.571544,3.577469
1,886.0,1983.0,4.472189,0.640801,3.610862
2,886.0,1984.0,4.567035,0.440832,3.694748
3,886.0,1985.0,4.885006,0.547543,3.796566
4,886.0,1986.0,4.999058,0.547543,3.976419


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4072 entries, 0 to 4071
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       4072 non-null   float32
 1   year     4072 non-null   float32
 2   sales    4072 non-null   float32
 3   labor    4072 non-null   float32
 4   capital  4072 non-null   float32
dtypes: float32(5)
memory usage: 111.3 KB


In [127]:
dummies = pd.get_dummies(df['year']).rename(columns=lambda x: 'year_' + str(x))
df = pd.concat([df, dummies], axis=1)
df = df.set_index(['id', 'year'])

In [134]:
exog_ind = ['labor', 'capital'] + [col for col in df.columns if 'year' in col]

exog_var = df[exog_ind]
endog_var = df['sales']



model_OLS = sm.OLS(endog=endog_var, exog=exog_var).fit()

In [129]:
print(model_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                 1.425e+04
Date:                Tue, 06 Jun 2023   Prob (F-statistic):               0.00
Time:                        10:49:38   Log-Likelihood:                -1527.7
No. Observations:                4072   AIC:                             3075.
Df Residuals:                    4062   BIC:                             3139.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
labor           0.5579      0.010     56.761      

In [130]:
R = np.zeros_like(res.params)
R[0:2] = [1, 1]
print(R)
q = 1

[1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [131]:
T_test = res.t_test((R, q))

In [132]:
T_test

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.9902      0.003     -3.049      0.002       0.984       0.996

In [133]:
model_FE = PanelOLS(dependent=endog_var, exog=exog_var, entity_effects=True).fit()

print(model_FE.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  sales   R-squared:                        0.7379
Estimator:                   PanelOLS   R-squared (Between):              0.9537
No. Observations:                4072   R-squared (Within):               0.7379
Date:                Tue, Jun 06 2023   R-squared (Overall):              0.9498
Time:                        10:49:39   Log-likelihood                    2302.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1111.5
Entities:                         509   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(9,3554)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):          6.321e+05
                            

In [135]:
formula = 'labor + capital = 1'

model_FE.wald_test(formula=formula)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 121.3166
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7fc7a7368af0